In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import fsolve
from __future__ import division
import matplotlib.pyplot as plt

### Peng Robinson Function

In [2]:
def PR(v, P, T, Pc, Tc, w):
    # in K, atm, l/gmol    
    R = 0.08206
    s = 0.37363 + (1.54226 - 0.26992*w)*w
    alpha = (1 + s *(1-(T/Tc)**0.5)) ** 2
    a = 0.45724 * alpha * (R * Tc) ** 2 / Pc
    b = 0.07780 * (R * Tc / Pc)
    return v**3 + (b - R*T/P)*v**2 - (3*b**2 + (2*b*R*T-a)/P)*v + (b**3 - a*b/P + b**2*R*T/P) 

## SRK

In [3]:
def SRK(v, P, T, Pc, Tc, w):
    # in K, atm, l/gmol
    R = 0.08206
    s = 0.48 + 1.574*w - 0.176*w**2
    alpha = (1 + s*(1 - (T/Tc)**0.5))**2
    a = 0.42748*R**2*Tc**2*alpha/Pc
    b = 0.08664*R*Tc/Pc
    return v**3 - (R*T/P)*v**2 + ((a-R*T*b)/P - b**2)*v - a*b/P 

## Vander Vaal

In [4]:
def VW(v, P, T, Pc, Tc, w = 1):
    # in K, atm, l/gmol
    R=0.08206
    a=27*R**2*Tc**2/(64*Pc)
    b=R*Tc/(8*Pc)
    return v**3 - (b + R*T/P)*v**2 + (a/P)*v - a*b/P

### We will take our initial guess from the ideal gas law 

In [5]:
def ideal(P, T):
    R = 0.08206
    v_ideal = R*T/P
    return v_ideal

## Sub-Critical 

In [6]:
def sub_critical(P, T, Pc, Tc, EOS, w=1):
    R = 0.08206
    b = 0.07780 * (R * Tc / Pc)
    if EOS == "PR":
        vsc = fsolve(PR, b, args = (P, T, Pc, Tc, w))
    elif EOS == "SRK":
        vsc = fsolve(SRK, b, args = (P, T, Pc, Tc, w))
    else:
        vsc = fsolve(VW, b, args = (P, T, Pc, Tc, w))
    return vsc

## Super-critical

In [7]:
def super_critical(P, T, Pc, Tc, EOS, w=1):
    v_guess = ideal(P, T)
    if EOS == "PR":
        vsc = fsolve(PR, v_guess,args = (P, T, Pc, Tc, w))
    elif EOS == "SRK":
        vsc = fsolve(SRK, v_guess,args = (P, T, Pc, Tc, w))
    else:
        vsc = fsolve(VW, v_guess,args = (P, T, Pc, Tc, w))
    return vsc  

## Solver

In [8]:
def VSOLVE(P, T, Pc, Tc, EOS, w=1):
    if T<Tc and P<Pc:
        v_sub = sub_critical(P, T, Pc, Tc, EOS, w)
    else:
        v_sub = None
    
    v_sup = super_critical(P, T, Pc, Tc, EOS, w)
    
    return v_sub, v_sup

## Water

In [12]:
data = pd.read_csv("water.csv")
data = np.array(data)

m  = np.shape(data)[0]
vpr_water = np.zeros((m, 2))
vsrk_water = np.zeros((m, 2))
vvw_water = np.zeros((m, 2))

v_exp_g = np.array(data[:,0])
v_exp_f = np.array(data[:,6])
#print(pd.DataFrame(data))

v_exp_g = pd.DataFrame((v_exp_g))
v_exp_f = pd.DataFrame((v_exp_f))


v_exp = np.array(np.column_stack((v_exp_f,v_exp_g)))

for d in range(len(v_exp)):
    if v_exp[d,0] == "None":
        v_exp[d,0] = None
    else:    
        v_exp[d,0] = np.float(v_exp[d,0])

v_exp = pd.DataFrame(v_exp)


Pres  = data[:,1]
Temp  = data[:,2]
Presc = data[:,3]
Tempc = data[:,4]
omega = data[:,5]


m  = np.shape(data)[0]

for d in range(m):
    vpr_water[d] = VSOLVE(Pres[d], Temp[d], Presc[d], Tempc[d], "PR", omega[d])   
for d in range(m):
    vsrk_water[d] = VSOLVE(Pres[d], Temp[d], Presc[d], Tempc[d], "SRK", omega[d])
for d in range(m):  
    vvw_water[d] = VSOLVE(Pres[d], Temp[d], Presc[d], Tempc[d], "VW", omega[d])
   
   

dpr_error_water = 1/v_exp - 1/vpr_water
dpr_error_water = (dpr_error_water)/(1/v_exp)*100

dsrk_error_water = 1/v_exp - 1/vsrk_water
dsrk_error_water = dsrk_error_water/(1/v_exp)*100

dvw_error_water = 1/v_exp - 1/vvw_water
dvw_error_water = dvw_error_water/(1/v_exp)*100

#V_WATER = pd.concat([Pres,Temp, v_exp, vpr_water, vpr_error_water, vsrk_water,vsrk_error_water, vvw_water, vvw_error_water], axis  =1)   
V_WATER = np.column_stack((Pres,Temp, v_exp, vpr_water, vsrk_water, vvw_water, dpr_error_water, dsrk_error_water, dvw_error_water))
V_WATER = pd.DataFrame(V_WATER)
names = ['Pres', 'Temp', 'v_exp_f', 'v_exp_g', 'vpr_water_f', 'vpr_water_g', 'vsrk_water_f', 'vsrk_water_g', 'vvw_water_f', 'vvw_water_g', 'dpr_error_water_f', 'dpr_error_water_g', 'dsrk_error_water_f', 'dsrk_error_water_g', 'dvw_error_water_f', 'dvw_error_water_g']
V_WATER.columns = names

/home/adi/.local/lib/python3.5/site-packages/scipy/optimize/minpack.py:163: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


In [13]:
V_WATER

,Pres,Temp,v_exp_f,v_exp_g,vpr_water_f,vpr_water_g,vsrk_water_f,vsrk_water_g,vvw_water_f,vvw_water_g,dpr_error_water_f,dpr_error_water_g,dsrk_error_water_f,dsrk_error_water_g,dvw_error_water_f,dvw_error_water_g
0,0.009869,280.15,0.018000,2325.600000,0.021011,2328.959336,0.023575,2328.961929,0.035908,2329.164297,14.329710,0.144242,23.648908,0.144353,49.872197,0.153029
1,0.019738,290.65,0.018018,1205.820000,0.021149,1207.948175,0.023741,1207.951486,0.036205,1208.139472,14.804496,0.176181,24.106871,0.176455,50.233417,0.191987
2,0.098692,318.95,0.018180,264.060000,0.021559,264.860125,0.024233,264.865016,0.037063,265.019941,15.672633,0.302094,24.977260,0.303934,50.948979,0.362215
3,0.986923,372.75,0.018774,30.492000,0.022526,30.728242,0.025387,30.735122,0.038993,30.844356,16.655590,0.768809,26.048444,0.791022,51.853205,1.142367
4,1.973847,393.35,0.019098,15.942600,0.022981,16.109570,0.025928,16.116966,0.039869,16.213094,16.896470,1.036467,26.340917,1.081879,52.097589,1.668366
5,3.947693,416.75,0.019512,8.323200,0.023573,8.440445,0.026628,8.448307,0.040985,8.531775,17.227638,1.389083,26.724545,1.480851,52.392345,2.444682
6,5.921540,431.95,0.019818,5.680800,0.024009,5.775346,0.027143,5.783454,0.041795,5.859777,17.456752,1.637068,26.985520,1.774966,52.582669,3.054325
7,7.895386,443.55,0.020070,4.325400,0.024374,4.407707,0.027572,4.415976,0.042466,4.487336,17.659413,1.867337,27.208468,2.051105,52.739169,3.608734
8,9.869233,453.05,0.020286,3.499200,0.024697,3.570994,0.027950,3.579381,0.043056,3.646961,17.860799,2.010489,27.421522,2.240077,52.884763,4.051631
9,19.738465,485.55,0.021186,1.792620,0.026000,1.841200,0.029471,1.849891,0.045412,1.906223,18.516601,2.638491,28.111387,3.095926,53.347602,5.959580


In [14]:
V_WATER.to_csv("v_water.csv")

## Carbon Dioxide

In [19]:
data = pd.read_csv("co2.csv")
data = np.array(data)
m  = np.shape(data)[0]
vpr_co2 = np.zeros((m, 2))
vsrk_co2 = np.zeros((m, 2))
vvw_co2 = np.zeros((m, 2))

v_exp_g = np.array(data[:,0])
v_exp_f = np.array(data[:,6])
#print(pd.DataFrame(data))

v_exp_g = pd.DataFrame((v_exp_g))
v_exp_f = pd.DataFrame((v_exp_f))


v_exp = np.array(np.column_stack((v_exp_f,v_exp_g)))

for d in range(len(v_exp)):
    if v_exp[d,0] == "None":
        v_exp[d,0] = None
    else:    
        v_exp[d,0] = np.float(v_exp[d,0])

v_exp = pd.DataFrame(v_exp)


Pres  = data[:,1]
Temp  = data[:,2]
Presc = data[:,3]
Tempc = data[:,4]
omega = data[:,5]


m  = np.shape(data)[0]

for d in range(m):
    vpr_co2[d] = VSOLVE(Pres[d], Temp[d], Presc[d], Tempc[d], "PR", omega[d])   
for d in range(m):
    vsrk_co2[d] = VSOLVE(Pres[d], Temp[d], Presc[d], Tempc[d], "SRK", omega[d])
for d in range(m):  
    vvw_co2[d] = VSOLVE(Pres[d], Temp[d], Presc[d], Tempc[d], "VW", omega[d])
   
   

dpr_error_co2 = 1/v_exp - 1/vpr_co2
dpr_error_co2 = (dpr_error_co2)/(1/v_exp)*100

dsrk_error_co2 = 1/v_exp - 1/vsrk_co2
dsrk_error_co2 = (dsrk_error_co2)/(1/v_exp)*100

dvw_error_co2 = 1/v_exp - 1/vvw_co2
dvw_error_co2 = (dvw_error_co2)/(1/v_exp)*100


#V_WATER = pd.concat([Pres,Temp, v_exp, vpr_water, vpr_error_water, vsrk_water,vsrk_error_water, vvw_water, vvw_error_water], axis  =1)   
V_CO2 = np.column_stack((Pres,Temp, v_exp, vpr_co2, vsrk_co2, vvw_co2, dpr_error_co2, dsrk_error_co2, dvw_error_co2))
V_CO2 = pd.DataFrame(V_CO2)
names = ['Pres', 'Temp', 'v_exp_f', 'v_exp_g', 'vpr_co2_f', 'vpr_co2_g', 'vsrk_co2_f', 'vsrk_co2_g', 'vvw_co2_f', 'vvw_co2_g', 'dpr_error_co2_f', 'dpr_error_co2_g', 'dsrk_error_co2_f', 'dsrk_error_co2_g', 'dvw_error_co2_f', 'dvw_error_co2_g']
V_CO2.columns = names

/home/adi/.local/lib/python3.5/site-packages/scipy/optimize/minpack.py:163: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
/home/adi/.local/lib/python3.5/site-packages/scipy/optimize/minpack.py:163: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  warnings.warn(msg, RuntimeWarning)


In [20]:
V_CO2

,Pres,Temp,v_exp_f,v_exp_g,vpr_co2_f,vpr_co2_g,vsrk_co2_f,vsrk_co2_g,vvw_co2_f,vvw_co2_g,dpr_error_co2_f,dpr_error_co2_g,dsrk_error_co2_f,dsrk_error_co2_g,dvw_error_co2_f,dvw_error_co2_g
0,19.7385,253.65,0.042724,0.837452,0.0416026,0.835368,0.047222,0.847831,0.0716375,0.904861,-2.69538,-0.249462,9.52515,1.2242,40.3609,7.44963
1,20.7254,255.25,0.043076,0.795256,0.0419923,0.792857,0.0476638,0.805319,0.0723073,0.861375,-2.58074,-0.302577,9.62542,1.24959,40.4265,7.67603
2,21.7123,256.79,0.043384,0.756756,0.0423825,0.754004,0.0481057,0.766464,0.07298,0.821599,-2.36304,-0.364929,9.81519,1.26656,40.5536,7.89234
3,22.6992,258.29,0.043692,0.721424,0.0427781,0.718438,0.048553,0.730893,0.0736664,0.785135,-2.13628,-0.415627,10.0118,1.2955,40.6894,8.11461
4,31.5815,269.96,0.046508,0.498476,0.0465096,0.493988,0.0527403,0.506336,0.0804148,0.553962,0.00346609,-0.908425,11.8169,1.5523,42.1649,10.0162
5,32.5685,271.1,0.046816,0.480832,0.0469529,0.476221,0.0532342,0.48855,0.081274,0.535551,0.291584,-0.968293,12.0565,1.57979,42.3973,10.2173
6,33.5554,272.22,0.047168,0.464112,0.0474061,0.459459,0.0537385,0.471769,0.0821761,0.518152,0.502339,-1.01282,12.2267,1.62296,42.6013,10.4294
7,34.5423,273.31,0.047476,0.448316,0.0478646,0.443559,0.0542477,0.455849,0.0831123,0.50164,0.811775,-1.07242,12.483,1.65253,42.8773,10.6299
8,35.5292,274.38,0.047828,0.433312,0.0483329,0.428496,0.0547673,0.440764,0.0841045,0.485971,1.04462,-1.12397,12.6705,1.6908,43.1326,10.8359
9,47.3723,285.75,0.052316,0.29788,0.0548604,0.293016,0.0619388,0.304946,0.0971502,0.343902,4.63802,-1.66009,15.536,2.31713,46.1494,13.3823


In [21]:
V_CO2.to_csv("v_co2.csv")

## Mixture

In [56]:
def VW_mix(v, P, T, Pc, Tc, x):
    # in K, atm, l/gmol
    R = 0.08206
    Pc_c = 72.80532939
    Tc_c = 304.13
    Pc_w = 217.7547496
    Tc_w = 647.1
    a_c = 27*R**2*Tc_c**2/(64*Pc_c)
    a_w = 27*R**2*Tc_w**2/(64*Pc_w)
    a = a_c*x**2 + 2*x*(1-x)*((a_c*a_w)**0.5) + a_w*(1-x)**2
    b_c = R*Tc_c/(8*Pc_c)
    b_w = R*Tc_w/(8*Pc_w)
    b = b_c*x + b_w*(1-x)
    return v**3 - (b + R*T/P)*v**2 + (a/P)*v - a*b/P  

In [57]:
def SRK_mix(v, P, T, Pc, Tc, x):
    # in K, atm, l/gmol
    R = 0.08206
    
    Pc_c = 72.80532939
    Tc_c = 304.13
    
    Pc_w = 217.7547496
    Tc_w = 647.1
    
    w_c = 0.268
    w_w = 0.334
    
    s_c = 0.48 + 1.574*w_c - 0.176*w_c**2
    alpha_c = (1 + s_c*(1 - (T/Tc_c)**0.5))**2
    a_c = 0.42748*R**2*Tc_c**2*alpha_c/Pc_c

    s_w = 0.48 + 1.574*w_w - 0.176*w_w**2
    alpha_w = (1 + s_w*(1 - (T/Tc_w)**0.5))**2
    a_w = 0.42748*R**2*Tc_w**2*alpha_w/Pc_w
    
    a = a_c*x**2 + 2*x*(1-x)*((a_c*a_w)**0.5) + a_w*(1-x)**2
    
    b_c = 0.08664*R*Tc_c/Pc_c
    b_w = 0.08664*R*Tc_w/Pc_w
    b = b_c*x + b_w*(1-x)
    
    return v**3 - (R*T/P)*v**2 + ((a-R*T*b)/P - b**2)*v - a*b/P 

In [58]:
def PR_mix(v, P, T, Pc, Tc, x):
    # in K, atm, l/gmol
    R = 0.08206
    
    Pc_c = 72.80532939
    Tc_c = 304.13
    
    Pc_w = 217.7547496
    Tc_w = 647.1
    
    w_c = 0.268
    w_w = 0.334
    
    s_c = 0.37363 + (1.54226 - 0.26992*w_c)*w_c
    alpha_c = (1 + s_c *(1-(T/Tc_c)**0.5)) ** 2
    a_c = 0.45724 * alpha_c * (R * Tc_c) ** 2 / Pc_c
    
    s_w = 0.37363 + (1.54226 - 0.26992*w_w)*w_w
    alpha_w = (1 + s_w *(1-(T/Tc_w)**0.5)) ** 2
    a_w = 0.45724 * alpha_w * (R * Tc_w) ** 2 / Pc_w
    
    a = a_c*x**2 + 2*x*(1-x)*((a_c*a_w)**0.5) + a_w*(1-x)**2
    
    b_c = 0.07780 * (R * Tc_c / Pc_c)
    b_w = 0.07780 * (R * Tc_w / Pc_w)
    b = b_c*x + b_w*(1-x)
    
    return v**3 + (b - R*T/P)*v**2 - (3*b**2 + (2*b*R*T-a)/P)*v + (b**3 - a*b/P + b**2*R*T/P) 

In [59]:
def sub_critical(P, T, Pc, Tc, EOS, x):
    R = 0.08206
    b = 0.07780 * (R * Tc / Pc)
    if EOS == "PR":
        vsc = fsolve(PR, b, args = (P, T, Pc, Tc, x))
    elif EOS == "SRK":
        vsc = fsolve(SRK, b, args = (P, T, Pc, Tc, x))
    else:
        vsc = fsolve(VW, b, args = (P, T, Pc, Tc, x))
    return vsc

In [60]:
def super_critical(P, T, Pc, Tc, EOS, x):
    v_guess = ideal(P, T)
    if EOS == "PR":
        vsc = fsolve(PR, v_guess,args = (P, T, Pc, Tc, x))
    elif EOS == "SRK":
        vsc = fsolve(SRK, v_guess,args = (P, T, Pc, Tc, x))
    else:
        vsc = fsolve(VW, v_guess,args = (P, T, Pc, Tc, x))
    return vsc  

In [61]:
def VSOLVE(P, T, Pc, Tc, EOS, x):
    if T<Tc and P<Pc:
        v_sub = sub_critical(P, T, Pc, Tc, EOS, x)
    else:
        v_sub = None
    
    v_sup = super_critical(P, T, Pc, Tc, EOS, x)
    
    return v_sub, v_sup

In [16]:
data = pd.read_csv("mix.csv")
data = np.array(data)
m  = np.shape(data)[0]
vpr_mix = np.zeros((m, 1))
vsrk_mix = np.zeros((m, 1))
vvw_mix = np.zeros((m, 1))

v_exp = np.array(data[:,0])

v_exp = pd.DataFrame(v_exp)

print(v_exp)
Pres  = data[:,1]
Temp  = data[:,2]
Presc = data[:,3]
Tempc = data[:,4]
x = data[:,5]


m  = np.shape(data)[0]

for d in range(m):
    v_sub, vpr_mix[d] = VSOLVE(Pres[d], Temp[d], Presc[d], Tempc[d], "PR", x[d])   
for d in range(m):
    v_sub, vsrk_mix[d] = VSOLVE(Pres[d], Temp[d], Presc[d], Tempc[d], "SRK", x[d])
for d in range(m):  
    v_sub, vvw_mix[d] = VSOLVE(Pres[d], Temp[d], Presc[d], Tempc[d], "VW", x[d])
   
   
vpr_mix = pd.DataFrame(vpr_mix)
vsrk_mix = pd.DataFrame(vsrk_mix)
vvw_mix = pd.DataFrame(vvw_mix)
print(vpr_mix)

dpr_error_mix = 1/v_exp - 1/vpr_mix
dpr_error_mix = (dpr_error_mix)/(1/v_exp)*100

dsrk_error_mix = 1/v_exp - 1/vsrk_mix
dsrk_error_mix = (dsrk_error_mix)/(1/v_exp)*100

dvw_error_mix = 1/v_exp - 1/vvw_mix
dvw_error_mix = (dvw_error_mix)/(1/v_exp)*100

V_MIX = np.column_stack((Pres,Temp, v_exp, x, vpr_mix, vsrk_mix, vvw_mix, dpr_error_mix, dsrk_error_mix, dvw_error_mix))
V_MIX = pd.DataFrame(V_MIX)
names = ['Pres', 'Temp', 'v_exp', 'x_c02', 'vpr_mix_g', 'vsrk_mix_g', 'vvw_mix_g', 'dpr_error_mix_g', 'dsrk_error_mix_g', 'dvw_error_mix_g' ]
V_MIX.columns = names

            0
0    0.142226
1    0.125399
2    0.117337
3    0.110955
4    0.096498
5    0.088117
6    0.075737
7    0.069234
8    8.185048
9    5.548897
10  17.812611
11  12.074378
12   2.246691
13   4.888685
14  10.643960
            0
0    0.131530
1    0.115507
2    0.109884
3    0.107977
4    0.096989
5    0.091326
6    0.079197
7    0.072722
8    8.170281
9    5.534352
10  17.796561
11  12.058763
12   2.235208
13   4.880622
14  10.642656


/home/adi/.local/lib/python3.5/site-packages/scipy/optimize/minpack.py:163: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
/home/adi/.local/lib/python3.5/site-packages/scipy/optimize/minpack.py:163: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  warnings.warn(msg, RuntimeWarning)


In [17]:
V_MIX

,Pres,Temp,v_exp,x_c02,vpr_mix_g,vsrk_mix_g,vvw_mix_g,dpr_error_mix_g,dsrk_error_mix_g,dvw_error_mix_g
0,246.138663,673.15,0.142226,0.10,0.131530,0.138515,0.130913,-8.131844,-2.679139,-8.641476
1,295.484826,673.15,0.125399,0.20,0.115507,0.121802,0.104000,-8.563252,-2.953055,-20.575752
2,344.830989,673.15,0.117337,0.30,0.109884,0.115819,0.094030,-6.782613,-1.310836,-24.786643
3,394.177153,673.15,0.110955,0.40,0.107977,0.113721,0.089899,-2.758282,2.432333,-23.421366
4,492.770787,673.15,0.096498,0.50,0.096989,0.102349,0.080714,0.505855,5.716661,-19.555521
5,591.463114,673.15,0.088117,0.60,0.091326,0.096442,0.076822,3.513682,8.632074,-14.703054
6,788.847767,673.15,0.075737,0.70,0.079197,0.083867,0.070183,4.369253,9.694201,-7.914059
7,986.232420,673.15,0.069234,0.80,0.072722,0.077036,0.067665,4.796514,10.127817,-2.318947
8,3.947989,398.15,8.185048,0.75,8.170281,8.181583,8.190420,-0.180738,-0.042343,0.065589
9,5.792943,398.15,5.548897,0.75,5.534352,5.545630,5.554422,-0.262805,-0.058908,0.099479


In [18]:
V_MIX.to_csv("v_mix.csv")